<a href="https://colab.research.google.com/github/Ian326/TI3001C/blob/main/E5_Construcci%C3%B3n_y_transformaci%C3%B3n_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recolección y descripción de datos
---
***Equipo:***

**Ruth Jiménez Vázquez - A01351923**

**José Ignacio Gómez Moreno - A01067362**

**Harry Hernández Grande - A01736341**

**Ian Joab Padrón Corona - A01708940**

---



## Carga de datos

In [ ]:
import pandas as pd
import numpy as np
import missingno as msno

In [ ]:
print(pd.read_csv('../content/vista_repdetail_tec.csv', low_memory=False).info())
# Como se puede observar, el csv no contiene el nombre de las columnas a utilizar
# Por tanto se debera asignar manualmente el nombre de las columnas
# segun el diccionario de datos proporcionado por el cliente.

In [ ]:
# Columnas dadas en el Diccionario de Datos proporcionado
columns = [
    "Column", "Taller", "OrderID", "kMS", "OpenedDate", "ClosedDate", "Status",
    "lagdias", "laghoras", "ClosedDay", "ClosedMonth", "ClosedYear", "OpenedDay",
    "OpenedMonth", "OpenedYear", "CreatedBy", "ClosedBy", "RepReason",
    "OrderType", "Supplier", "COMPCDKEY", "COMPCODE", "DESCRIP", "QTYRCVD",
    "QTYREQD", "CHGAMT", "TAXAMT", "SUBTOTAL", "TOTAL", "UnitID", "UnitType",
    "UnitYear", "EC", "Flota", "ReasonType", "COMPLAINT", "Jobcode", "TipoLinea",
    "NumParte", "FechaPromesa", "Estimate", "CompleteDay", "CompleteMonth",
    "CompleteYear", "CompleteDate", "MECHANIC", "CITA"
]

data = pd.read_csv('../content/vista_repdetail_tec.csv', names=columns, low_memory=False)
data.info()

In [ ]:
data.sample(3)

## Documentacion de los esquemas de datos

* ¿Cuál es nuestra fuente de datos?

Actualmente, la informacion es recabada por nuestro SocioFormador TDR. Tenemos acceso a un escritorio remoto que esta a su vez conectado con un servidor de Microsoft SQL Server.

A partir de dicho servidor, llamamos a una de las vistas a las que nos dieron acceso, para recabar toda la informacion sobre las reparaciones de su flota.


* ¿Cómo obtenemos la información?

A traves de una Query de SQL a la que nos autorizaron, que seria la siguiente:


```
SELECT * FROM [TMWSuite].[dbo].[vista_repdetail_tec]
```




## Diccionario de datos

| Column        | Descripción                                                                                       |
| ------------- | ------------------------------------------------------------------------------------------------- |
| OrderID       | ID de la orden de reparación                                                                      |
| Taller        | Acrónimo del taller donde se realizó la reparación                                                |
| kMS         | Kilometros viajados por la unidad                                                                     |
| OpenedDate    | Fecha de apertura de la orden de reparación                                                       |
| ClosedDate    | Fecha de cierre de la orden de reparación                                                         |
| Status        | Estatus de la orden de reparación                                                                 |
| lagdias       | Días entre cierre y apertura de la orden de reparación                                            |
| laghoras      | Horas entre cierre y apertura de la orden de reparación                                           |
| ClosedDay     | Día del mes en que se cerró la orden de reparación                                                |
| ClosedMonth   | Mes que se cerró la orden de reparación                                                           |
| ClosedYear    | Año en que se cerró la orden de reparación                                                        |
| OpenedDay     | Día del mes en que se abrió la orden de reparación                                                |
| OpenedMonth   | Mes que se abrió la orden de reparación                                                           |
| OpenedYear    | Año en que se abrió la orden de reparación                                                        |
| CreatedBy     | Usuario que creó la orden de reparación                                                           |
| ClosedBy      | Usuario que cerro la orden de reparación                                                          |
| RepReason     | Razón de reparación                                                                               |
| OrderType     | Tipo de orden. Standar es en uno de nuestros talleres propios. Vendor es con un proveedor externo |
| Supplier      | Proveedor que realizó la reparación                                                               |
| COMPCDKEY     | Codigo de componente (No aporta info importante)                                                  |
| COMPCODE      | Codigo utilizado para identificar el tipo de reparación                                           |
| DESCRIP       | Descripción de la reparación                                                                      |
| QTYRCVD       | Cantidad de piezas o trabajo recibidas                                                            |
| QTYREQD       | Cantidad de piezas o trabajo requeridas                                                           |
| CHGAMT        | Costo unitario                                                                                    |
| TAXAMT        | IVA                                                                                               |
| SUBTOTAL      | Subtotal de la reparación (No incluye IVA)                                                        |
| TOTAL         | Total de la reparación (Incluye IVA)                                                              |
| UnitID        | Numero de unidad                                                                                  |
| UnitType      | Tipo de unidad (Tracto, Caja Seca, Dolly, etc.)                                                   |
| UnitYear      | Año del modelo de la unidad                                                                       |
| EC            | Equipo administrativo que se encarga de la operación de un conjunto de proyectos                  |
| Flota         | Nombre del proyecto en que trabaja el tracto. (Cliente)                                           |
| ReasonType    | Razón (No aporta info importante)                                                                 |
| COMPLAINT     | Queja reportada                                                                                   |
| Jobcode       | Numero para identificar el tipo de reparación                                                     |
| TipoLinea     | Describe el tipo de línea en la orden de reparación (Piezas, mano de obra, servicios, etc.)       |
| NumParte      | Numero de parte descarga en la línea de reparación                                                |
| FechaPromesa  | Fecha de promesa para terminar las reparaciones de la unidad                                      |
| Estimate      | Monto estimado (No lo utilizamos)                                                                 |
| CompleteDay   | Día del mes en que se completó la orden                                                           |
| CompleteMonth | Mes que se completó la orden de reparación                                                        |
| CompleteYear  | Año en que se completó la orden de reparación                                                     |
| CompleteDate  | Fecha en que se completó la orden de reparación                                                   |
| MECHANIC      | Mecánico interno que realizó la reparación                                                        |
| CITA          | Indica si la unidad tenía una cita agenda previamente                                             |

## Analisis de la estructura de los datos

* ¿Que tipo de datos tenemos? **Tenemos 13 de tipo float, 4 int, y 30 object.**
* ¿Requerimos modificar el tipo de datos de alguna columna? **Si, para ser precisos, debemos convertir las siguientes columnas:**
  * **A int64**:
    * lagdias: Son cantidad de dias, no pueden venir con decimales
    * laghoras: Son cantidad de horas, no pueden venir con decimales
    * ClosedDay: Es el numero del dia en que fue cerrada (puede ser de 1 - 31)
    * QTYRCVD: Es una cantidad de piezas, no puede venir con decimales
    * QTYREQD: Es una cantidad de piezas, no puede venir con decimales
    * CompleteDay: Es el numero del dia en que fue cerrada (puede ser de 1 - 31)
    * CompleteYear: Es un año (no puede tener punto decimal o ser un string)
  
  * **A datetime64**
    * OpenedDate: Actualmente es un object, debe ser fecha
    * ClosedDate: Actualmente es un object, debe ser fecha
    * FechaPromesa: Actualmente es un object, debe ser fecha
    * CompleteDate: Actualmente es un object, debe ser fecha

* ¿Existen nulos? **Si, actualmente hay 689,643 registros faltantes en toda la informacion que tenemos**

In [ ]:
data.dtypes.head(10)

In [ ]:
data.isnull().sum()

In [ ]:
data.isnull().sum().sum() # <- Cantidad de nulos

## Perfiles involucrados

*   Equipo de TDR que nos puede ayudar con los datos (Identificar por nombre y cargo)


### Ing. Emmanuel Vargas Pérez

**Jefe de Administración de Equipo**

Nos proporcionará información sobre el proyecto que desarrollaremos (Sayer), incluyendo configuraciones de las flotas de tractocamiones y las fallas más comunes que suelen presentar estos vehículos.






### Ing. Mauricio Romo Ávila

**Subdirector de Mantenimiento**

Nos brindará información sobre el mantenimiento de los tractocamiones, incluyendo las fallas comunes, los costos asociados a estas fallas, las ubicaciones donde ocurren con mayor frecuencia, el número de talleres disponibles (tanto propios como generales), y responderá dudas sobre sus mantenimientos correctivos.


### Ing. Brandon Velazco

**Jefe de Taller**

Aportará información relevante sobre los tractocamiones, incluyendo detalles sobre las partes y componentes específicos que suelen fallar, en qué consisten estas fallas, así como la duración y el costo de las mismas.

*   Los profesores nos brindarán las herramientas y conocimientos
necesarios para lograr las metas y objetivos.

## Limpieza de Indices y Columnas

* ¿Que indices vamos a utilizar?
 **La mejor opcion seria el OrderID, ya que es un valor unico para identificar cada orden de mantenimiento**

* ¿Que columnas requieren un cambio de Tipo de Dato?
  * **A int64**:
      * lagdias: Son cantidad de dias, no pueden venir con decimales
      * laghoras: Son cantidad de horas, no pueden venir con decimales
      * ClosedDay: Es el numero del dia en que fue cerrada (puede ser de 1 - 31)
      * QTYRCVD: Es una cantidad de piezas, no puede venir con decimales
      * QTYREQD: Es una cantidad de piezas, no puede venir con decimales
      * CompleteDay: Es el numero del dia en que fue cerrada (puede ser de 1 - 31)
      * CompleteYear: Es un año (no puede tener punto decimal o ser un string)

    * **A float64**:
      * Estimate: Se encuentra como object actualmente, debe ser un precio en decimal
    
    * **A datetime64**
      * OpenedDate: Actualmente es un object, debe ser fecha
      * ClosedDate: Actualmente es un object, debe ser fecha
      * FechaPromesa: Actualmente es un object, debe ser fecha
      * CompleteDate: Actualmente es un object, debe ser fecha

In [ ]:
# Nombrando columna 'OrderID' como índice
data.set_index('OrderID', inplace=True)
data

In [ ]:
# Convirtiendo tipos de datos a DateTime
data['OpenedDate'] = pd.to_datetime(data['OpenedDate'])
data['ClosedDate'] = pd.to_datetime(data['ClosedDate'])
data['FechaPromesa'] = pd.to_datetime(data['FechaPromesa'])
data['CompleteDate'] = pd.to_datetime(data['CompleteDate'])

In [ ]:
# Redondeando todos los decimales a enteros
data['lagdias'] = data['lagdias'].round(0)
data['laghoras'] = data['laghoras'].round(0)
data['ClosedDay'] = data['ClosedDay'].round(0)
data['ClosedYear'] = data['ClosedYear'].round(0)
data['QTYRCVD'] = data['QTYRCVD'].round(0)
data['QTYREQD'] = data['QTYREQD'].round(0)
data['CompleteDay'] = data['CompleteDay'].round(0)
data['CompleteYear'] = data['CompleteYear'].round(0)

In [ ]:
# Convirtiendo datos nulos a 0s
data['lagdias'] = data['lagdias'].fillna(0)
data['laghoras'] = data['laghoras'].fillna(0)
data['ClosedDay'] = data['ClosedDay'].fillna(0)
data['ClosedYear'] = data['ClosedYear'].fillna(0)
data['QTYRCVD'] = data['QTYRCVD'].fillna(0)
data['QTYREQD'] = data['QTYREQD'].fillna(0)
data['CompleteDay'] = data['CompleteDay'].fillna(0)
data['CompleteYear'] = data['CompleteYear'].fillna(0)

In [ ]:
# Convertir tipos de datos a enteros
data['lagdias'] = data['lagdias'].astype(int)
data['laghoras'] = data['laghoras'].astype(int)
data['ClosedDay'] = data['ClosedDay'].astype(int)
data['ClosedYear'] = data['ClosedYear'].astype(int)
data['QTYRCVD'] = data['QTYRCVD'].astype(int)
data['QTYREQD'] = data['QTYREQD'].astype(int)
data['CompleteDay'] = data['CompleteDay'].astype(int)
data['CompleteYear'] = data['CompleteYear'].astype(int)

In [ ]:
# Convertir tipos de datos a float
# data['Estimate'] = data['Estimate'].astype(float)
# No se puede, porque la informacion en la columna 'Estimate' no es realmente un estimado

In [ ]:
# Volver a revisar el tipo de dato de las columnas
data.info()

## Analisis de los datos (Descriptiores Estadisticos)

In [ ]:
# Seleccionar sólo las variables relevantes para las estadisticas
data_descript = data[['lagdias', 'laghoras','TOTAL','UnitYear',
                      'RepReason', 'UnitType', 'TipoLinea', 'CITA']]

data_descript.sample(10)

**Estadística descriptiva de las variables numéricas**

In [ ]:
# Estadística descriptiva de las variables numéricas
data_descript.describe()

| Variable numérica | Descripción | Rango | Desviación estándar |
|-----------|-----------|-----------|-----------|
| lagdias    | Días entre cierre y apertura de la orden de reparación   | 655    | Datos no tan dispersos    |
| laghoras    | Horas entre cierre y apertura de la orden de reparación    | 15,720    | Datos muy dispersos    |
| TOTAL    | Total de la reparación (Incluye IVA)   | 1,086,544   | Datos demasiado dispersos   |
| UnitYear    | Año del modelo de la unidad   | 116   | Datos no dispersos   |




*   **lagdias**: Los días de apertura y cierre de una reparación están ligeramente dispersos
*   **laghoras**: Las horas de apertura y cierre de una reparación presentan un gran rango de diferencia entre el valor máximo y mínimo, además de que están muy dispersas.
*   **TOTAL**: El costo total de las reparaciones tienen el rango más amplio, y con una alta desviación estándar.
*   **UnitYear**: Al tratarse de los años del modelo de cada unidad, no se presenta tanta dispersión.





In [ ]:
msno.matrix(data_descript[['lagdias', 'laghoras', 'TOTAL', 'UnitYear']])
#No hay notoria presencia de valores null

In [ ]:
data_descript.boxplot(figsize = (20,30))
#El costo total de una reparación presenta la mayor cantidad de outliers, valores extremos
#Es importante consultar si estos valores son correctos, o pudiera tratarse de un error, de lo contrario se deberían eliminarse o imputarse

#El resto de variables no presentan tantos valores extremos

In [ ]:
#Histograma para analizar la variable más dispersa, TOTAL

data_descript.hist(['TOTAL'], figsize = (10,10))

**Estadística descripctiva de variables categóricas**

In [ ]:
# Estadística descripctiva de variables categóricas
data_descript.describe(include='object')

In [ ]:
# Valores más repetidos en las variables cuantitativas
dataquant = data_descript.select_dtypes(include=object)

for i in range(dataquant.shape[1]):
  print(dataquant.iloc[:,i].value_counts().head(5))
  print()

Identificar si hay columnas comunes en los dataframes empleados que permitan “enlazar” los registros en las siguientes fases.

**Debido a que solo utilizamos un dataframe para esta parte inicial del proyecto, no contamos con columnas por las que se pueda enlazar a otros dataframes. A futuro sería posible que tengamos más de uno que sí valdría la pena enlazar a traves de llaves o columnas en común.**

# **Selección y limpieza de datos**

## Eliminar las columnas no relevantes

Columna|Justificacion
------|--------------
Column| No sabemos que es
Taller| Viene implicito en el OrderID (nuestro index)
ClosedDate| Tenemos una metrica mas precisa (laghoras)
ClosedDay| ""
ClosedMonth| ""
ClosedYear| ""
OpenedDay| Tenemos una metrica mas general (OpenedDate)
OpenedMonth| ""
OpenedYear| ""
CreatedBy| No afecta realmente quien inicio la orden de reparacion
ClosedBy| No afecta realmente quien cerro la orden de reparacion
OrderType| Solo existen 2 tipos 'Standard' & 'Vendor'. Asumimos que no tiene efecto en la reparacion
Supplier| No afecta realmente de donde se obtuvo la pieza de reparacion
**COMPCDKEY**| **No sabemos si tiene utilidad o no**
**COMPCODE**| **No sabemos si tiene utilidad o no**
DESCRIP| Tiene informacion demasiado especifica para cada caso, no nos seria util
QTYRCVD| No afecta realmente si se daño 1 o mas piezas, si no, que fallaron
QTYREQD| Similar al anterior.
CHGAMT| Lo importante es la metrica del costo por reparacion (TOTAL)
TAXAMT| ""
SUBTOTAL| ""
EC| No afecta realmente el equipo administrativo del proyecto
ReasonType| Segun el diccionario de datos, no aporta informacion relevante
FechaPromesa| No genera impacto en la reparacion
Estimate| " "
CompleteDay| Tenemos una metrica mas precisa (laghoras)
CompleteMonth| ""
CompleteYear| ""
CompleteDate| ""
MECHANIC| Asumimos que no genera impacto en la reparacion quien la hizo

In [ ]:
# Seleccionar sólo las columnas relevantes para las estadisticas
data_filtered_cols = data.drop(['Column', 'Taller', 'ClosedDate', 'Status',
                   'ClosedDay', 'ClosedMonth', 'ClosedYear', 'OpenedDay', 'OpenedMonth', 'OpenedYear',
                   'CreatedBy', 'ClosedBy', 'OrderType', 'Supplier', 'COMPCDKEY', 'COMPCODE',
                   'DESCRIP', 'QTYRCVD', 'QTYREQD', 'CHGAMT', 'TAXAMT', 'SUBTOTAL', 'EC',
                   'ReasonType', 'FechaPromesa', 'Estimate', 'CompleteDay',
                   'CompleteMonth', 'CompleteYear', 'CompleteDate', 'MECHANIC'], axis=1)

data_filtered_cols.sample(10)

In [ ]:
data_filtered_cols.info()

In [ ]:
data_filtered_cols.isnull().sum()

## Limpieza de las columnas (Trim)

Notamos que algunas de las columnas extraidas desde el archivo .csv tenian espacios en blanco que deben ser removidos para evitar complicaciones mas adelante en la agrupacion y filtrado de informacion. Es por eso que se procedera a arreglar dichos errores en la captura de la informacion

In [ ]:
data_filtered_cols.info()

In [ ]:
data_filtered_cols.head(5)

In [ ]:
# Trim 'RepReason' column
data_filtered_cols['RepReason'] = data_filtered_cols['RepReason'].str.strip()
# Verificar que se haya hecho correctamente
data_filtered_cols['RepReason'].value_counts().index[0:5]

In [ ]:
# Trim 'UnitType' column
data_filtered_cols['UnitType'] = data_filtered_cols['UnitType'].str.strip()
# Verificar que se haya hecho correctamente
data_filtered_cols['UnitType'].value_counts().index

In [ ]:
# Trim 'Flota' column
data_filtered_cols['Flota'] = data_filtered_cols['Flota'].str.strip()
# Verificar que se haya hecho correctamente
data_filtered_cols['Flota'].value_counts().index[0:5]

In [ ]:
# Trim 'COMPLAINT' column
data_filtered_cols['COMPLAINT'] = data_filtered_cols['COMPLAINT'].str.strip()
# Verificar que se haya hecho correctamente
data_filtered_cols['COMPLAINT'].value_counts().index[0:5]

In [ ]:
# Trim 'Jobcode' column
data_filtered_cols['Jobcode'] = data_filtered_cols['Jobcode'].str.strip()
# Verificar que se haya hecho correctamente
data_filtered_cols['Jobcode'].value_counts().index[0:5]

In [ ]:
# Trim 'TipoLinea' column
data_filtered_cols['TipoLinea'] = data_filtered_cols['TipoLinea'].str.strip()
# Verificar que se haya hecho correctamente
data_filtered_cols['TipoLinea'].value_counts().index

In [ ]:
# Trim 'NumParte' column
data_filtered_cols['NumParte'] = data_filtered_cols['NumParte'].str.strip()
# Verificar que se haya hecho correctamente
data_filtered_cols['NumParte'].value_counts().index[0:5]

In [ ]:
# Trim 'CITA' column
data_filtered_cols['CITA'] = data_filtered_cols['CITA'].str.strip()
# Verificar que se haya hecho correctamente
data_filtered_cols['CITA'].value_counts().index

Para la ultima columna 'CITA' se observa que existen registros con un campo vacio, por lo que se procede a sustituir este campo N/A por un texto de referencia (DESCONOCIDO)

In [ ]:
# Sustituir valores de 'CITA' con un string vacio por 'Desconocido'
data_filtered_cols['CITA'] = data_filtered_cols['CITA'].replace('', 'DESCONOCIDO')
# Verificar que se haya hecho correctamente
data_filtered_cols['CITA'].value_counts().index

## Restablecer los índices con identificadores adecuados.

La columna OrderID ya ha sido colocada como el índice del dataframe, y de esta manera resulta útil para el análisis.

## Seleccionar o filtrar los registros u observaciones de interés
Con la finalidad de operar dataframes de menor complejidad.

Dado el proyecto asignado al equipo, nos centraremos en la informacion sobre todas las reparaciones de la flota 'Sayer', por lo que deberemos filtrar dicha informacion.

In [ ]:
# Extraemos la informacion del proyecto Sayer Full
data_sayer = data_filtered_cols[data_filtered_cols['Flota'] == 'Sayer Full'][:]
data_sayer.info()

## Identificar valores faltantes y eliminarlos
(Tambien se pueden imputar según la naturaleza de la omisión. Justificar cada caso.)

In [ ]:
data_sayer[['kMS', 'TipoLinea', 'CITA', 'NumParte']].isnull().sum()

In [ ]:
filas_null = data_sayer[data_sayer.isnull().any(axis=1)]
filas_null

In [ ]:
data_sayer['CITA'] = data_sayer['CITA'].fillna('DESCONOCIDO')
data_sayer['CITA'].isnull().sum()

In [ ]:
data_sayer['TipoLinea'] = data_sayer['TipoLinea'].fillna('DESCONOCIDO')
data_sayer['TipoLinea'].isnull().sum()

In [ ]:
data_sayer['NumParte'] = data_sayer['NumParte'].fillna('DESCONOCIDO')
data_sayer['NumParte'].isnull().sum()

In [ ]:
data_sayer.isnull().sum()

## Detectar valores atípicos para su posterior transformación.


In [ ]:
data_sayer.boxplot(vert=False)
#Observamos que en la columna de TOTAL es la que aparentemente hay mas datos
#atipicos, por lo que vamos a anilizarla a detalle


In [ ]:
data_sayer[['TOTAL']].boxplot(vert=False)

In [ ]:
data_sayer.describe()

In [ ]:
data_sayer[data_sayer['TOTAL'] > 30000]

In [ ]:
# Masks
bajos = data_sayer['TOTAL'] <= 0 #consideramos que no es congruente que la columna de total tenga valores negativos

altos = data_sayer['TOTAL'] >= 30000 #consideramos que es posbile que una reparacion sea
#bastante costosa de manera que consideramos mantener los datos elevados, exeptuando,
#los que ya estan muy alejados que podemos ver en la grafica que son mas de 30000


valores_atipicos = bajos | altos
valores_atipicos.sum()

In [ ]:
data_sayer_clean = data_sayer[~valores_atipicos]
data_sayer_clean.info()

In [ ]:
data_sayer_clean[['TOTAL']].boxplot(vert=False)

In [ ]:
data_sayer_clean[['laghoras']].boxplot(vert=False)

In [ ]:
data_sayer_clean[data_sayer_clean['laghoras'] < 0]

In [ ]:
data_sayer_clean['laghoras'].describe()

In [ ]:
bajos = data_sayer_clean['laghoras'] <= 0 #consideramos que no es congruente
#que la columna de lagdias tenga valores negativos

altos = data_sayer_clean['laghoras'] >= 2259 #la reparacion es probable que
#lleve bastantes dias, de manera que consideramos dejar los datos de dias mas grandes


valores_atipicos = bajos | altos
valores_atipicos.sum()

In [ ]:
data_sayer_clean = data_sayer_clean[~valores_atipicos]
data_sayer_clean.info()

In [ ]:
data_sayer_clean[['laghoras']].boxplot(vert=False)

In [ ]:
data_sayer_clean[['lagdias']].boxplot(vert=False) #Al eliminar lo negativos de dias se eliminaron tambien
#los de horas

In [ ]:
data_sayer_clean['lagdias'].describe()

In [ ]:
data_sayer_clean.describe() #eliminamos los valores que consideramos atipicos

In [ ]:
data_sayer_clean[['kMS']].boxplot(vert=False)

In [ ]:
bajos = data_sayer_clean['kMS'].isna()

valores_atipicos = bajos
valores_atipicos.sum()

In [ ]:
data_sayer_clean = data_sayer_clean.fillna(data_sayer_clean['kMS'].median())
data_sayer_clean.info()

In [ ]:
data_sayer_clean['kMS'].describe()

In [ ]:
data_sayer = data_sayer_clean

## Eliminar registros duplicados.

In [ ]:
# Cantidad de elementos en el df
data_sayer.index.size

In [ ]:
# Cantidad de elementos en el df si se eliminan las filas con informacion identica
data_sayer.drop_duplicates().index.size

Haciendo una resta simple, podemos calcular que actualmente existen $6389-6072 = 317$ duplicados. Esto añade un gran sesgo al análisis posterior, por lo que se procede a eliminarlos.

In [ ]:
data_sayer.drop_duplicates(inplace=True)

In [ ]:
data_sayer.info()

# **Construcción y transformación de datos**

## Integrar los dataframes
Creando los enlaces o combinaciones relevantes al dominio o contexto del proyecto.

Debido a que sólo se ha estado utilizando un sólo dataframe para el proyecto, es que no es necesario realizar ningún enlade al dominio. Cabe resaltar que esta misma base de datos se ha sometido a distintas limpiezas y procesamientos para cumplir con los objetivos indicados en el entregable.

## Agregación y agrupamiento.
 Construir nuevas características o variables con las tecnicas descritas

Primero, vamos a agrupar los valores de la columna 'COMPLAINT' para separar los siguientes tipos de
mantenimiento:
* PREVENTIVO
* CORRECTIVO
* DESCONOCIDO

In [ ]:
#Analizar los tipos de reparaciones que se realizan a los tractos más comunes
data_sayer['COMPLAINT'].value_counts().index

In [ ]:
# Añadir una nueva columna para identificar claramente los tipos de 'COMPLAINT'
# El valor por defecto de la columna sera 'CORRECTIVO' debido a que es el tipo de reparacion mas comun
data_sayer['ComplaintType'] = 'CORRECTIVO'

# Establecer en 'COMPLAINT' = 'PREVENTIVO' para los casos especficos que se pueden determinar
ComplaintTypePreventive = ['PM', 'PREVENT VENC', 'GASTADO', 'DESGASTADO', 'SUCIO', 'BAJA PRESION', 'PRESIÓN BAJA', 'GOLPETEA']
data_sayer.loc[data_sayer['COMPLAINT'].isin(ComplaintTypePreventive), 'ComplaintType'] = 'PREVENTIVO'
# Excepto para los casos que no se pueden determinar por ser demasiado ambiguos.
# Estos se clasificarán como 'DESCONOCIDO'
ComplaintTypeUnknown = ['DAÑO OP', 'SE NEUTRALIZ', 'DESGASTE IRR', 'JALA DERECHA', 'SIN DIESEL', 'ACORTADO', 'OTROS', 'OTHER']
data_sayer.loc[data_sayer['COMPLAINT'].isin(ComplaintTypeUnknown), 'ComplaintType'] = 'DESCONOCIDO'

In [ ]:
# Muestreamos para verificar que el agrupamiento se haya hecho correctamente
data_sayer['ComplaintType'].value_counts()

In [ ]:
#¿Qué porcentaje representan los mantenimiento preventivos segun el ComplaintType respecto a todas las operaciones de mantenimiento?
percent = (data_sayer.groupby('ComplaintType')['ComplaintType'].count() / data_sayer.shape[0] * 100).round(2)
# Ordenar por porcentaje
percent = percent.sort_values(ascending=False)

# Imprimir los porcentajes de mantenimientos por tipo
for i in range(len(percent)):
    print(f" (Complaint Type) Los mantenimientos de tipo '{percent.index[i]}' representan el {percent.iloc[i]}% de todas las operaciones de mantenimiento.")


Posteriormente, vamos a agrupar los valores de la columna 'RepReason' para identificar los tipos de
mantenimiento:
* PREVENTIVO
* CORRECTIVO
* DESCONOCIDO

In [ ]:
#Analizar los tipos de reparaciones que se realizan a los tractos más comunes
data_sayer['RepReason'].value_counts().index

In [ ]:
# Añadir una nueva columna para identificar claramente los tipos de 'COMPLAINT'
# El valor por defecto de la columna sera 'DESCONOCIDO' debido a que es el tipo de reparacion mas comun
# Lamentablemente, es muy ambigua la informacion en esta columna, por lo que tendremos que clasificarla como 'DESCONOCIDO'
data_sayer['RepReasonType'] = 'DESCONOCIDO'

# Establecer en 'COMPLAINT' = 'PREVENTIVO' para los casos especificos que se pueden determinar
ComplaintTypePreventive = ['PM', 'DESGASTE', 'LAVADO', 'FALTA PRESIÓ']
data_sayer.loc[data_sayer['RepReason'].isin(ComplaintTypePreventive), 'RepReasonType'] = 'PREVENTIVO'
# Establecer en 'COMPLAINT' = 'CORRECTIVO' para los casos especificos que se pueden determinar
ComplaintTypeCorrective = ['MC', 'PERFORADA', 'PONCHADURA', 'FUGA AIRE', 'FOCOS', 'FUGA ACEITE', 'CAMBIO DAÑO', 'ROBO', '7 VIAS DAÑAD', 'BATERÍA DAÑA', 'GOLPE', 'CORTO', 'FALLA GPS']
data_sayer.loc[data_sayer['RepReason'].isin(ComplaintTypeCorrective), 'RepReasonType'] = 'CORRECTIVO'

In [ ]:
# Muestreamos para verificar que el agrupamiento se haya hecho correctamente
data_sayer['RepReasonType'].value_counts()

In [ ]:
#¿Qué porcentaje representan los mantenimiento preventivos segun el ComplaintType respecto a todas las operaciones de mantenimiento?
percent = (data_sayer.groupby('RepReasonType')['RepReasonType'].count() / data_sayer.shape[0] * 100).round(2)
# Ordenar por porcentaje
percent = percent.sort_values(ascending=False)

# Imprimir los porcentajes de mantenimientos por tipo
for i in range(len(percent)):
    print(f" (RepReason Type) Los mantenimientos de tipo '{percent.index[i]}' representan el {percent.iloc[i]}% de todas las operaciones de mantenimiento.")

## Discretización o binning
Reagrupar atributos de alta cardinalidad mediante los metodos descritos

In [ ]:
#Elegimos el metodo de discretizacion qcut ya que no tenemos rangos bien definidos
#de manera que con este metodo dividimos las variables en intervalos iguales,
#a lo cual tomamos 3 criterios, Reparacion_rapida, Reparacion_Normal y
#Reparacion_lenta con la finalidad de hacer mas descriptiva la variable de horas
# y tener nocion si la reparacion tomo demasiado tiempo o fue muy lenta
#Decidimos eligir la variable de horas, porque al analizar esta variable, observamos
#que es mas exacta que la variable dias, ya que esta ultima son dias enteros.

data_sayer['Clasif_rep'], cut_bin = pd.qcut(x=data_sayer['laghoras'],
                                    q=3,
                                    labels=['Reparacion_rapida','Reparacion_Normal','Reparacion_lenta'],
                                    retbins=True)
cut_bin

In [ ]:
data_sayer['Clasif_rep'].value_counts()
#Observamos que lo dividio casi exactamente en 3 partes iguales

In [ ]:
#Similar al caso anterior, al no tener unos rangos bien definidos,
#nos decidimos por el metodo de qcut para dividirlo en 3 partes,Costo_bajo
# Costo_normal, Costo_elevado .

data_sayer['Clasif_costo'], cut_bin = pd.qcut(x=data_sayer['TOTAL'],
                                    q=3,
                                    labels=['Costo_bajo','Costo_normal','Costo_elevado'],
                                    retbins=True)
cut_bin

In [ ]:
data_sayer['Clasif_costo'].value_counts()

## One-hot encoding
Para las variables **categóricas** que consideren necesarias.

In [ ]:
data_sayer['CITA'] = data_sayer['CITA'].str.strip()
print(list(data_sayer['CITA'].value_counts().index))

In [ ]:
data_sayer['CITA'].replace([''], 'DESCONOCIDO', inplace=True)

In [ ]:
!pip install category_encoders

In [ ]:
col_cat = data_sayer.select_dtypes(include=['object', 'category'])
col_cat.head(5)

In [ ]:
from category_encoders import CountEncoder
# codificación por conteo
data_sayer = CountEncoder(cols=['CITA']).fit(data_sayer).transform(data_sayer)

In [ ]:
from category_encoders import OneHotEncoder
# codificación por OneHot
data_sayer = OneHotEncoder(cols=['TipoLinea']).fit(data_sayer).transform(data_sayer)

In [ ]:
from category_encoders import OneHotEncoder
# codificación por OneHot
data_sayer = OneHotEncoder(cols=['UnitType']).fit(data_sayer).transform(data_sayer)

In [ ]:
data_sayer.rename(columns={'UnitType_1': 'TRACTOR', 'UnitType_2': 'TRAILER', 'UnitType_3': 'DOLLY', 'TipoLinea_1': 'PART', 'TipoLinea_2': 'SERVICES', 'TipoLinea_3': 'LABOR'}, inplace = True)
data_sayer

Para la parte de 'One-Hot Encoding' se decidio codificar unicamente 3 variables. 'CITA', 'TipoLinea', 'UnitType'.

Para las variables 'TipoLinea' y 'UnitType' se decidio codificar con OneHot ya que de esta manera será mucho mas facil encontrar y leer que tipo de unidad de esta manejando y que tipo de linea.

La variable 'CITA' se codificó con Count ya que de esta manera se podrá conocer la cantidad de registros por cada situación de las citas y así tener un mayor control.

## Feature transformation
Para las variables con valores atípicos identificados.

In [ ]:
# Verificamos que columnas vamos a trabajar
# (Deben ser numericas, para poder transformarlas)
data_sayer.info()

In [ ]:
# Analizamos cual es el valor minimo para cada una, es muy importante
# para obtener el metodo para transformar
data_sayer.describe()

In [ ]:
# Importamos una libreria para la visualizacion de la distribucion de los datos en nuestras columnas
# nominales
import matplotlib.pyplot as plt

In [ ]:
pd.plotting.scatter_matrix(data_sayer[['lagdias', 'laghoras', 'TOTAL', 'UnitYear']],
                           figsize=(12,10))
plt.show()

# Aqui debemos identificar aquellas que no tienen una distribucion normal, y que tenga sentidos aplicarlas
# una transformacion. En este caso, podriamos ocupar las variables 'dias, horas y TOTAL'

In [ ]:
# Dada la naturaleza de nuestros datos, no seria posible aplicar una 'Transformada
# Logaritmica', porque contamos con valores entre 0 y 1.
# Ademas si intentamos utilizar un metodo como el de 'Box-Cox', nos daria un error
# en la ejecucion, pues este solo permite valores mayores a 1.
# Es por ello que nuestro unico metodo de transformacion plausible para utilizar
# seria el de Yeo-johnson, que acepta un rango de valores de negativos, positivos
# y 0.

import seaborn as sns
from sklearn.preprocessing import PowerTransformer

scaler = PowerTransformer(method='yeo-johnson')

pow_trans = scaler.fit_transform(data_sayer[['lagdias', 'laghoras', 'TOTAL']])
data_sayer['pow_lagdias'] = pow_trans[:, 0]
data_sayer['pow_laghoras'] = pow_trans[:, 1]
data_sayer['pow_total'] = pow_trans[:, 2]

In [ ]:
# Comparativa entre las distribuciones anteriores y posteriores a la transformacion
fig, ax = plt.subplots(1, 2)
ax[0].hist(data_sayer.lagdias)
ax[0].set_title('LAGDIAS')
ax[1].hist(np.log(data_sayer.pow_lagdias))
ax[1].set_title('Pow LAGDIAS')

In [ ]:
# Comparativa entre las distribuciones anteriores y posteriores a la transformacion
fig, ax = plt.subplots(1, 2)
ax[0].hist(data_sayer.laghoras)
ax[0].set_title('LAGHORAS')
ax[1].hist(np.log(data_sayer.pow_laghoras))
ax[1].set_title('Pow LAGHORAS')

In [ ]:
# Comparativa entre las distribuciones anteriores y posteriores a la transformacion
fig, ax = plt.subplots(1, 2)
ax[0].hist(data_sayer.TOTAL)
ax[0].set_title('TOTAL')
ax[1].hist(np.log(data_sayer.pow_total))
ax[1].set_title('Pow TOTAL')

In [ ]:
# Finalmente hacemos una comparativa con su covarianza, para afirmar el incremento
# entre la variabilidad entre una y otra
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,6))
sns.heatmap(data_sayer[['lagdias', 'laghoras', 'TOTAL']].corr(), annot=True, ax=ax1).set_title('Original data')
sns.heatmap(data_sayer[['pow_lagdias', 'pow_laghoras', 'pow_total']].corr(), annot=True, ax=ax2).set_title('Pow Transformed data')

# RESULTADOS OBTENIDOS

En este entregable se logró encontrar el porcentaje significativo del número de rescates en carretera respecto al total.

Además, se clasificaron las reparaciones realizadas acorde a su periodo de tiempo, y los costos de las mismas según su rango.

Por último, se trabajaron los valores atípicos afirmando el incremento entre la variabilidad de los valores importantes para el proyecto.

# Convertir a HTML

In [ ]:
%%shell
jupyter nbconvert --to html /content/E5_Construcción_y_transformación_de_datos.ipynb